# Load Library & Preprocess Data 

In [1]:
import os
GPU = f'0'
os.environ['CUDA_VISIBLE_DEVICES']=GPU
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
from sklearn.model_selection import train_test_split

from config import *
from Models.models import yoloV3, yoloLoss

### Set Seed

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

### Load Data 

In [3]:
df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [4]:
df.head()

,image_path,x_rankle,y_rankle,x_rknee,y_rknee,x_rhip,y_rhip,x_lhip,y_lhip,x_lknee,...,x_lshoulder,y_lshoulder,x_lelbow,y_lelbow,x_lwrist,y_lwrist,x_human,y_human,w_human,h_human
0,015601864.jpg,394,620,269,616,185,573,188,647,221,...,185,692,240,693,313,688,257,594,286,205
1,015601864.jpg,293,895,279,910,223,945,218,1012,315,...,206,924,203,1013,263,955,222,952,229,229
2,015599452.jpg,0,0,0,0,543,806,593,720,0,...,299,719,516,711,466,545,329,619,525,525
3,015599452.jpg,0,0,0,0,607,987,571,1194,0,...,269,1145,475,1226,433,1096,412,1010,533,533
4,015599452.jpg,0,0,0,0,537,228,536,74,0,...,251,26,423,26,0,0,315,133,506,506


### Split train & validation Data 

In [5]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=SEED)

In [6]:
train_df.shape, val_df.shape

((25994, 37), (2889, 37))

# Build Model 

In [7]:
with tf.device(f'/device:GPU:{GPU}'):
    model = yoloV3(size=size, training=True, classes=num_classes)
    print(model.summary())
    anchors = yolo_anchors
    anchor_masks = yolo_anchor_masks

Model: "yolov3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
yolo_darknet (Functional)       ((None, None, None,  40620640    input[0][0]                      
__________________________________________________________________________________________________
yolo_conv_0 (Functional)        (None, 13, 13, 512)  11024384    yolo_darknet[0][2]               
__________________________________________________________________________________________________
yolo_conv_1 (Functional)        (None, 13, 13, 256)  2759680     yolo_conv_0[0][0]                
_____________________________________________________________________________________________

# Compile Model 

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = [yoloLoss(anchors[mask], classes=num_classes) for mask in anchor_masks]

In [9]:
model.compile(optimizer=optimizer, loss=loss)

callbacks = [
    ReduceLROnPlateau(verbose=1),
    EarlyStopping(patience=3, verbose=1),
    ModelCheckpoint('{CKPT_PATH}/yolov3_train_{epoch}.tf',
                   verbose=1,
                   save_weights_only=True),
    TensorBoard(log_dir='logs')
]

# Training 

In [ ]:
# history = model.fit(train_dataset,
#                    epochs=EPOCHS,
#                    callbacks=callbacks,
#                    validation_data=val_dataset)